In [248]:
import pandas as pd 
import numpy as np 

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

import time
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from catboost import CatBoostClassifier

# import sys
# !{sys.executable} -m pip install catboost
# nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alena\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alena\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [249]:

train = pd.read_csv('data/train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [250]:
train

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


In [251]:
# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def clear(text):
        cleaned = re.sub(r'[^а-яА-я ]', ' ', text)
        cleaned_words = cleaned.split()
        cleaned_sentence = " ".join(cleaned_words)
        cleaned_sentence = cleaned_sentence.lower()
        return cleaned_sentence

def lemm(sentence):
        lemmas = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
        sentence = " ".join(lemmas)
        return sentence

stop_words = set(stopwords.words('russian')) 
print(stop_words)

def stop(sentence):
        word_tokens = word_tokenize(sentence) 
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        final = " ".join(filtered_sentence)
        return final

def total_clean(text):
    corpus_final = stop(clear(lemm(text)))
    return corpus_final

{'ну', 'из', 'и', 'я', 'был', 'чего', 'во', 'всего', 'ж', 'моя', 'том', 'все', 'здесь', 'куда', 'тот', 'ему', 'разве', 'будто', 'хоть', 'за', 'более', 'в', 'то', 'у', 'меня', 'тебя', 'всю', 'больше', 'быть', 'вы', 'чуть', 'всегда', 'где', 'этого', 'через', 'или', 'нее', 'мы', 'вот', 'кто', 'эту', 'два', 'над', 'будет', 'тогда', 'них', 'да', 'на', 'тут', 'всех', 'об', 'впрочем', 'со', 'есть', 'были', 'ли', 'того', 'ее', 'когда', 'не', 'совсем', 'ты', 'нет', 'до', 'много', 'он', 'так', 'чтоб', 'потому', 'мне', 'она', 'а', 'себе', 'уже', 'какая', 'там', 'они', 'к', 'их', 'можно', 'сам', 'никогда', 'было', 'как', 'между', 'лучше', 'раз', 'с', 'бы', 'только', 'по', 'его', 'даже', 'чтобы', 'такой', 'ведь', 'чем', 'другой', 'нас', 'им', 'конечно', 'мой', 'какой', 'от', 'сейчас', 'этом', 'про', 'о', 'вас', 'что', 'теперь', 'нибудь', 'вам', 'один', 'зачем', 'хорошо', 'нельзя', 'ним', 'еще', 'ни', 'него', 'уж', 'под', 'эти', 'может', 'ней', 'потом', 'была', 'этой', 'тем', 'же', 'но', 'ей', 'без'

In [252]:
#train = train.head(5000) # 1000 за 40 секунд

## УЖЕ ПОСЧИТАНО
# start = time.time()
# train_cleaned = train.apply(lambda x: total_clean(x['text']), axis = 1)
# end = time.time()

# print(len(train), end-start)
# cleaned = pd.DataFrame({'cleaned': train_cleaned})
# cleaned.to_csv('data/cleaned.csv', index=False)

In [253]:
all_categories = list(train['category'].unique())
category_index = {category: all_categories.index(category) for category in all_categories}
category_index

{'winter_sport': 0,
 'extreme': 1,
 'football': 2,
 'boardgames': 3,
 'hockey': 4,
 'esport': 5,
 'athletics': 6,
 'motosport': 7,
 'basketball': 8,
 'tennis': 9,
 'autosport': 10,
 'martial_arts': 11,
 'volleyball': 12}

In [254]:
train['text'] = pd.read_csv('data/cleaned.csv')
train['category'] = train.apply(lambda x: category_index[x['category']], axis = 1)
train = train.dropna()
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38740 entries, 0 to 38739
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       38740 non-null  int64 
 1   category  38740 non-null  int64 
 2   text      38740 non-null  object
dtypes: int64(2), object(1)
memory usage: 908.1+ KB


,oid,category,text
0,365271984,0,волшебные фото виктория поплавская евгениямедв...
1,503385563,1,возвращение подземелье треша эйфория тупости ж...
2,146016084,2,лучшие чешские вратари доминик доминатор гашек...
3,933865449,3,валрак решил подкормить сильно свежими слухами...
4,713550145,4,шестеркин затаскивает рейнджерс финал восточно...


In [255]:
train['category'].value_counts().sort_values()

0     2800
8     2850
2     2860
4     2950
12    2950
6     2970
5     2990
9     3000
3     3020
7     3030
11    3050
1     3110
10    3160
Name: category, dtype: int64

In [256]:
train, valid_test = train_test_split(train, test_size = 0.001)
valid, test = train_test_split(valid_test, test_size = 0.5)
print('Длина: тренировчоной {}, тестовой {}, валидационной {} выборок'.format(len(train), len(test), len(valid)))

# 

Длина: тренировчоной 38701, тестовой 20, валидационной 19 выборок


In [257]:
corpus_train = list(train['text'])
corpus_valid = list(valid['text'])
corpus_test = list(test['text'])

target_train = list(train['category'])
target_valid = list(valid['category'])
target_test = list(test['category'])

In [258]:
#мешок слов
count_vect = CountVectorizer()
bow_train = count_vect.fit_transform(corpus_train)
bow_valid = count_vect.transform(corpus_valid)
bow_test = count_vect.transform(corpus_test)

#tf-idf
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [259]:
model = LogisticRegression(solver = 'sag', n_jobs = -1, random_state = 123)
model.fit(bow_train, target_train)
predictions = model.predict(bow_test)

predictions2 = model.predict_proba(bow_test)

av='micro'
print('f1:',f1_score(target_test, predictions, average=av), 
      ', precision:',precision_score(target_test, predictions, average=av),
      ', recall:', recall_score(target_test, predictions, average=av))


f1: 0.75 , precision: 0.75 , recall: 0.75


C:\Users\alena\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [260]:
real_test = pd.read_csv('data/test.csv')

## УЖЕ ПОСЧИТАНО
# real_test_cleaned = real_test.apply(lambda x: total_clean(x['text']), axis = 1)
# cleaned = pd.DataFrame({'cleaned': real_test_cleaned})
# cleaned.to_csv('data/cleaned_test.csv', index=False)

In [261]:
real_test['text'] = pd.read_csv('data/cleaned_test.csv')
real_test.info()
real_test.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26260 entries, 0 to 26259
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   oid     26260 non-null  int64 
 1   text    26260 non-null  object
dtypes: int64(1), object(1)
memory usage: 410.4+ KB


,oid,text
0,749208109,спочно сообщество продается р покупкой пишите ...
1,452466036,естественное восстановление тяжелой тренировки...
2,161038103,тема нарядов продолжается одна британских журн...
3,663621910,привет избранный спрашиваешь происходит ответ ...
4,566255305,король пятисотников деле андрей рублев успешно...


In [262]:
bow_real_test = count_vect.transform(real_test['text'])

In [263]:
predictions2 = model.predict_proba(bow_real_test)

av='micro'

results = []

for row in predictions2:
    maximum = np.max(row)
    index_of_maximum = np.argmax(row)
    
    if maximum > 0.69:
        results.append(index_of_maximum)
    else:
        results.append(None)
    print(maximum, index_of_maximum)

0.5076352754948708 5
0.41553668696078083 12
0.777204351269618 9
0.9000806313671672 5
0.8276185516294136 9
0.6765178912773371 12
0.7290130727431645 10
0.9999999999967293 12
0.9765425972510723 3
0.11794832020476065 7
0.9997890935989717 6
0.6570442601055912 5
0.9886388551547486 3
0.9048472834520023 11
0.8024131651143893 0
0.9999998744371488 4
0.9999999562743159 3
0.1917156632064437 2
0.9987353257417686 8
0.9710793570279852 9
0.5837782647992091 12
0.14837694036395954 4
0.564541079679058 12
0.9991137535839317 6
0.9953071540728717 10
0.17838925150755214 1
0.9381405376189352 4
0.9963057532574937 3
0.8939736244017984 4
0.1291609293923896 1
0.9442443312172083 7
0.6141824021988869 0
0.9636892149931819 8
0.9482541969274997 0
1.0 10
0.28063723977514793 12
0.5813946618469468 7
0.9993993591125806 6
0.9938753885889245 0
0.9107126519391808 12
0.6722365905485068 0
0.9999999999978384 10
0.29481425294404356 0
0.9999999999991678 9
0.18401805100804608 1
0.8019588019509104 7
0.9990925529533558 8
0.896924965

0.9258686000761336 9
0.9991956375777415 9
0.9999169073816475 1
0.997212307358469 12
0.9946081821209978 7
0.2052824588101395 3
0.9999981965326927 10
0.7026160387012184 4
0.9994192217151789 10
0.2399898529272586 1
0.24473243605633754 5
0.6588084004083095 0
0.8692878160509732 4
0.9808680042234543 7
0.99993290431084 8
0.43781729212768566 0
0.9996417309485711 9
0.9993463779725281 9
0.5457716667697127 8
0.36413861626792043 4
0.19808777274458364 3
0.9998823231174587 8
0.9534251987870594 4
0.5846040139492786 2
0.18809338905709522 1
0.9772683717594441 9
0.9997172772395543 7
0.9159522624978166 0
0.9988371701934031 3
0.9303864827535673 3
0.9933203549068872 3
0.11524662239935392 5
0.9961290859314651 9
0.22201699747734754 1
0.22685488769554124 11
0.9999999990380288 6
0.3491989233691288 4
0.9924743974671911 8
0.8182387219570345 12
0.4254796951559686 4
0.9995252642137844 4
0.8529696014439858 1
0.29759543955602913 8
0.5209271211133885 1
0.819199891736312 3
0.9248292445758801 8
0.9963307477378269 1
0.6

0.49152336546983444 6
0.24823998065197095 5
0.3021749256142314 5
0.9807590794406473 0
0.9988448799145851 9
0.22447547327515519 11
0.9999997768676692 8
0.7366027345493712 7
0.3774351538763013 11
0.9993364789853303 6
0.7915181709640016 10
0.7975279818491235 2
0.16258681028084743 0
0.1583739997324127 12
0.5497824257693328 1
0.44155809168117893 1
0.6360693608674862 0
0.3075295917881363 0
0.7706400038428355 0
0.9996298314082035 0
0.30361825554663396 8
0.8464429985978926 3
0.9339702628652998 12
0.9996872906079819 3
0.9995581716973548 0
0.45138859464938635 1
0.702553986846835 3
0.3969648432306512 5
0.47069985205674103 2
0.4411137658928667 2
0.5651213198457199 2
0.1689464445138231 5
0.2652876568474677 5
0.16967278898575747 12
0.9713709592590289 4
0.5848728508870468 2
0.9948359938440758 7
0.7743426460450463 8
0.6957423487088814 10
0.2504490520606955 6
0.260384025436077 8
0.8760342322098729 5
0.9408828673021727 11
0.4102895872231341 5
0.9526089913018504 8
0.9978691040495581 0
0.5632403805478182 

0.20516075606726328 6
0.7712888256583671 2
0.9918914243059779 3
0.8098847165929252 9
0.9144032212754453 2
0.25234863516664274 7
0.9654289786990897 3
0.9959430920441513 7
0.12157567209305079 5
0.9994769223663584 8
0.6848472586166403 10
0.7993888280123813 0
0.11322020168081816 11
0.15433831508841328 4
0.9999997666705509 8
0.9702767728298145 9
0.9924743974671911 8
0.20138565863781785 5
0.2015578621092676 11
0.15225335067989085 11
0.9775535170950358 0
0.9999350607130406 9
0.253992329589561 1
0.9668854804203473 3
0.746692628175797 0
0.9999819014608444 6
0.3572328071776565 5
0.25251128480402285 12
0.16507292050884173 1
0.4393693930415088 6
0.9973284920581803 6
0.9999881966444725 0
0.9060893536768222 3
0.17182183782002713 11
0.9946203465474927 2
0.8435207915059747 1
0.9890106824008059 3
0.9785066436516267 8
0.9999977017355828 0
0.7740180799010717 10
0.9801522953701685 11
0.33472724616521904 1
0.9899113028616388 3
0.9761776903843549 4
0.9983898178884449 11
0.9949130638841105 8
0.99999864087678

0.13116070121969323 5
0.6643613482183973 5
0.6503461075077511 2
0.9999943193122343 11
0.887690179396099 9
0.2778906876882245 10
0.5242217611748416 9
0.9970168280404192 7
0.9999870715934245 6
0.1604184974725428 8
0.8189335069315217 8
0.31651264782418187 3
0.9965196799238528 9
0.9507101321883544 4
0.8355008701007315 0
0.8243113122345812 12
0.9999999820036708 7
0.9994786645631661 3
0.9475563636770659 4
0.892687083996883 10
0.9139960398563605 6
0.11822048317385705 5
0.22595751025186703 0
0.989818550706032 7
0.38598503399378764 11
0.6258662714418258 7
0.9785808803188727 6
0.966442822195322 9
0.4312773382971309 7
0.7973520776699573 4
0.4843441285469143 12
0.9918787344463972 9
0.9999997758746095 9
0.9582406150902663 0
0.9753990191561644 12
0.9949649978823334 12
0.9976719004102251 10
0.8647536425110869 8
0.9544945320263272 11
0.9993763732854875 8
0.9976283419368307 4
0.9974304792692473 9
0.9999985581171493 0
0.32052381627556853 4
0.9979980787559802 12
0.9999999835419171 11
0.46809940691494667 

...

0.9796817939546466 4
0.9983378069330383 6
0.5178147682629997 7
0.9758383526582196 4
0.9999998806947157 10
0.2311192556439321 5
0.4139082927729767 1
0.29346258354427635 2
0.9998720591844485 9
0.9984500792298341 12
0.157675973013056 2
0.5000697340040401 5
0.20757445764876836 8
0.9999998209697664 7
0.8330574442781694 12
0.2193211820141201 1
0.992343527555977 2
0.9984767515610435 4
0.9449605772135242 1
0.9999998191855235 11
0.8591281449250939 9
0.362584369607966 10
0.9999292754102532 6
0.7669850747000285 8
0.47899429767525653 3
0.9984430477174385 9
0.9988097937525942 11
0.11210725319632682 11
0.9897399004207142 6
0.9827670117555474 8
0.8996794010738927 10
0.7594967296708414 3
0.1784508289909098 2
0.3946681756977467 2
0.12829707418591976 2
0.9546835141779001 11
0.7990408012071598 4
0.9094433236939979 5
0.9421915690047866 9
0.6809350268962931 1
0.9999986789925317 3
0.43415673192252435 3
0.9987705267597058 4
0.9976608875413631 11
0.9741173535426978 6
0.919413386416113 10
0.9638688863276673 3


0.9972671827882275 6
0.30299095674145654 5
0.2378188068800027 8
0.5675129062748553 1
0.12034830768575898 5
0.9821932585652098 9
0.999571407471294 10
0.7729613935588986 7
0.1580129367195703 5
0.6561635625504469 10
0.5430117699736078 7
0.9563541560790982 10
0.9547697032724435 10
0.999997000852625 12
0.18528003759067743 4
0.9999209004444446 2
0.9990507283801187 6
0.9999997657757893 10
0.6225591282072866 3
0.9872086229220564 6
0.5217670250746982 2
0.538432579408212 6
0.9850194403869437 0
0.5030632973447986 0
0.28783655695907095 4
0.999928753995445 1
0.5040605149192383 7
0.9926853679901776 1
0.3227230080194888 4
0.6482463609288615 11
0.9989552071329567 0
0.48035090344213244 0
0.9816260081400175 7
0.9994420709337682 3
0.9915083380731778 12
0.4473545803682044 8
0.38422637077119887 11
0.3702205596042438 2
0.9585233338907665 0
0.4967682763513483 3
0.8796157661134817 1
0.9960421110543799 12
0.2848815634538685 1
0.7082463489988243 7
0.7208756376325236 3
0.9999891286212297 6
0.36885125429636123 5


0.21987632018456618 5
0.9999227243865695 8
0.8385188253589028 12
0.8111322268717933 5
0.9972941194996198 7
0.1933094143859092 11
0.7730281144496225 5
0.4358782886012696 3
0.9924743974671911 8
0.7755494155957986 8
0.37585515955231574 5
0.41277851028888446 10
0.9553198358234386 4
0.5593251222105925 1
0.9847750290777257 7
0.9687502377046141 5
0.9949160586195267 4
0.9990208652574906 3
0.5146860288193955 0
0.9999998722583169 0
0.5291955254944627 7
0.7115005042147134 8
0.9995735398364393 9
0.16573413334711695 11
0.7884744013395683 2
0.9999997693915618 10
0.6506792610104523 2
0.757225267111818 1
0.9999998988806664 4
0.9999609199358787 10
0.9999999717793006 6
0.9999944797339512 6
0.9715200239968081 2
0.9999999932351218 6
0.28341314399963263 0
0.8475022202138758 3
0.26622324692883603 12
0.8315731932236551 11
0.9990576957184377 9
0.9999999642529867 8
0.9364228634060088 12
0.4810053681992087 0
0.344860011661199 10
0.6849808517788406 12
0.8718493714445459 4
0.1703668199468967 1
0.8322799679618716 

0.9917152822202752 9
0.2414203308184404 11
0.9999722291851737 12
0.14194008482310674 12
0.9999806281534261 7
0.8452276516569415 7
0.9827126545114283 3
0.1734019414944684 11
0.8640927311890808 1
0.9900695763509246 1
0.6971409355944955 2
0.5327600421085843 12
0.2170632988161713 1
0.36751728040204407 12
0.9968686283527634 6
0.9996340147128242 3
0.9998057592606324 9
0.19286694160830925 12
0.8355458355292469 1
0.8577416104394299 1
0.28364395870045006 3
0.21200081868380521 5
0.9924743974671911 8
0.8654480040249728 12
0.28725702713433526 2
0.999935356557604 8
0.1749094104436388 1
0.9741092183459952 10
0.6900439021417664 7
0.9924743974671911 8
0.9999985854697813 11
0.7402710818095639 5
0.5461015632783321 7
0.9999997759505834 4
0.5046763226793545 4
0.1469896189960464 1
0.6693105068059574 11
0.9998529213325998 9
0.6665114343507887 11
0.6519643256399147 7
0.42872254554997413 2
0.987203450695202 6
0.9976623585498634 0
0.24630131384538095 6
0.3848299629958076 6
0.8194265240640333 12
0.9999355793366

0.3212289916765733 8
0.9998337698851562 4
0.9999996275953184 0
0.9573423983777697 9
0.7542080783564843 11
1.0 4
0.5888571595417842 7
0.7726698799733995 0
0.3951363228726544 5
0.8311408908971125 7
0.33365477599211724 6
0.31104523582479215 1
0.5509062060034927 1
0.6588084004083095 0
0.9979343302568711 3
0.8189031661061532 5
0.8279079063053689 11
0.999999103196622 7
0.9948715288441413 7
0.37025656790685246 5
0.3317853156569325 0
0.5355597199448239 2
0.2984743920961321 1
0.6218830950588449 0
0.1199114318703869 8
0.6610522699190327 11
0.6588084004083095 0
0.980930166311352 3
0.7916082616261932 0
0.9923846403088762 0
0.4596360882171759 11
0.667771795105611 1
0.9986854364556904 8
1.0 6
0.9976225455874539 3
0.9995021942289717 9
0.9999999999590281 2
0.2484130819917515 7
0.2852008377443184 7
0.7814000329608334 8
0.45073360425895653 11
0.8420153399613731 7
0.7611209463207476 5
0.3851648652608154 0
0.9999793286283922 2
0.37790153734428567 0
0.9433982321146747 8
0.9906700834135206 12
0.921974586116

In [264]:
real_test = pd.read_csv('data/test.csv')
real_test['category'] = results
real_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26260 entries, 0 to 26259
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   oid       26260 non-null  int64  
 1   text      26260 non-null  object 
 2   category  15397 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 615.6+ KB


In [265]:
real_test = real_test.dropna()
real_test['category'] = real_test['category'].astype(int)
real_test['category'] = real_test.apply(lambda x: list(category_index.keys())[list(category_index.values()).index(x['category'])], axis = 1)


answer = real_test.drop(['text'], axis=1)

In [266]:
answer = pd.DataFrame({'category': real_test['category'], 'oid':real_test['oid'] })
answer.to_csv('data/final_answer69_1.csv', index=False)
real_test.to_csv('data/my_full_answer69_1.csv', index=False)

In [109]:
model = LogisticRegression(solver ='newton-cg' , n_jobs = -1, random_state = 123) # 'sag'
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
av='micro'
print('f1:',f1_score(target_test, predictions, average=av), 
      ', precision:',precision_score(target_test, predictions, average=av),
      ', recall:', recall_score(target_test, predictions, average=av))

f1: 0.845360824742268 , precision: 0.845360824742268 , recall: 0.845360824742268


In [110]:
tf_idf_real_test = count_tf_idf.transform(real_test['text'])

In [111]:
predictions2 = model.predict_proba(tf_idf_real_test)

av='micro'

results = []

for row in predictions2:
    maximum = np.max(row)
    index_of_maximum = np.argmax(row)
    
    if maximum > 0.65:
        results.append(index_of_maximum)
    else:
        results.append(None)
    print(maximum, index_of_maximum)

0.5144550648512493 5
0.1519243285269508 11
0.3104476312155216 9
0.39859694201640916 5
0.6802500161796433 9
0.28627821075279336 12
0.2573327318074711 10
0.9844056108114881 12
0.7734107457769369 3
0.11638071037276827 11
0.6546992955694363 6
0.33916208891607064 5
0.7731844537962248 3
0.23049819266944976 11
0.6595469395678152 0
0.9840642254769214 4
0.9748954517115925 3
0.2537337826291991 2
0.7007951437271053 8
0.5211940921890172 9
0.2643527534530341 12
0.18699749486078185 0
0.27079232948796966 12
0.9975212381533998 6
0.9690603767161576 10
0.1680541212391386 1
0.6267400712780817 4
0.6016170134146162 3
0.7074184985419681 4
0.15939879738045307 4
0.8097714329453266 7
0.6761050278659799 0
0.26026290383479955 8
0.4307580988607367 0
0.9966470984072433 10
0.3043597557837754 12
0.49903524712478003 7
0.7065580170986553 6
0.8053986288081125 0
0.6524563948412869 12
0.4183952198593658 0
0.972564061257856 10
0.2916211485768751 0
0.9967442758883384 9
0.22957489070814344 1
0.41315395935361504 7
0.96418105

0.8508386393454699 8
0.4554758766682909 0
0.9607310652214588 9
0.999242742973131 9
0.3481141950839487 8
0.3813703847259262 4
0.19158328160828744 3
0.9242153526006883 8
0.4418009530589673 4
0.3727108052036901 2
0.2847800174176409 1
0.8192959527451256 9
0.8282478058699771 7
0.6975854730481345 0
0.7892373693085979 3
0.3964915698760346 3
0.851202690792243 3
0.16638836437622237 4
0.8967179625131196 9
0.21245260040287858 1
0.17652356535935096 0
0.9972866567871861 6
0.2838885658094539 4
0.9833026633504114 8
0.32140282013325566 12
0.3604190973989022 4
0.9117047184585174 4
0.47020779204810287 1
0.21236483722683952 8
0.37206776170127864 1
0.47453379457961453 3
0.8280388301468011 8
0.7741110884270395 1
0.3038788675363715 10
0.20489677562548883 1
0.5827176992467965 0
0.6097420900683341 8
0.7375128429495399 6
0.4123624365696842 0
0.30224376116844714 5
0.2659314417177759 4
0.5464712782943132 6
0.5096362180350201 6
0.5160692582835406 7
0.13202625955589342 7
0.3991684855490628 2
0.47096141613776565 3


0.7329929525655767 9
0.8492872793211793 12
0.18527584059941568 4
0.1568308677289037 4
0.3547806122942784 11
0.9261865445615184 8
0.8053157658927246 2
0.1895734588478713 5
0.8605001928356549 10
0.5069090226109828 3
0.878368728021659 4
0.17530486706119047 7
0.6373609501843063 8
0.853612486197717 8
0.15665687994775113 11
0.9325001213256179 6
0.8609038469141099 0
0.4701035108416153 8
0.4432890529653857 7
0.20927335499777117 7
0.600795160134007 2
0.9833026633504114 8
0.35398622335225616 5
0.8788327358208952 0
0.17545146031105835 0
0.9842395067735469 9
0.48542366946759363 10
0.19157665105711544 5
0.505928180333516 12
0.17675591447398617 11
0.5221935368814054 2
0.8375442975605496 0
0.27212058628578134 12
0.7410195513406624 0
0.22009780857563446 7
0.5849611461942257 0
0.2853645401022755 6
0.7031818812064442 3
0.9404811606142026 7
0.23363857426843762 5
0.2667250987329123 5
0.3760541576557014 2
0.8800486129693378 3
0.21110583104083122 1
0.9572019739657041 6
0.4557937983062515 2
0.758973007558210

0.6030165887609393 5
0.6189799612376512 12
0.502355959124742 8
0.13525016079600632 3
0.857496728709273 12
0.6033271937565285 3
0.41975391860262085 11
0.4215675830051335 5
0.9604842390396905 10
0.345478366637198 0
0.5073002345469548 5
0.9360650052881525 9
0.5172001113813486 9
0.855825715989615 8
0.10818611649274841 11
0.7134982542621193 3
0.6492103078230894 9
0.6686431733324604 5
0.7420105086739135 12
0.1582546138027677 5
0.9598901495581454 4
0.2603686037725321 1
0.12740238097894763 5
0.9176963190147733 12
0.21058735795977465 3
0.1596349443859141 3
0.1524693184651785 1
0.9709299928155476 6
0.257762842993186 2
0.15471019358051263 1
0.17427816439506683 12
0.5055371233806581 8
0.9942815323078203 6
0.11780472516193624 1
0.9970706334802762 6
0.5475226516628231 11
0.336800521293357 4
0.2882620475865108 10
0.3751175065397797 5
0.9339702427702194 1
0.4127439411610785 2
0.3726726596071832 0
0.9759658446725328 3
0.9833026633504114 8
0.8578370376621115 4
0.3073737486767756 3
0.8252861663418052 8
0

0.7765215995698095 3
0.9247083691164415 11
0.442287975261102 10
0.3798430719330377 4
0.8878147746105687 9
0.7349021623995591 9
0.13782399274222581 12
0.20188770927402921 1
0.44364949484272503 6
0.35246650343167923 8
0.21575338426432275 8
0.28547915544239505 5
0.6155176371930099 1
0.36325867501244063 1
0.44151981429939013 7
0.8270688853764308 6
0.39779774959816383 11
0.9756247070401141 2
0.4786527472042224 11
0.989800970934624 6
0.20786292464075357 1
0.11920214355728331 4
0.29424475884838347 1
0.7138499149126083 5
0.15553893418439454 1
0.8044300747429134 6
0.9279841156244657 4
0.16650870406365276 1
0.9631054849166258 4
0.3147749049848517 11
0.7233514551798901 8
0.7854614883847786 7
0.4872804566298766 9
0.17023155668694054 0
0.3836340390698557 9
0.25539426827907197 0
0.22109131991403472 5
0.21863574336977545 2
0.30275470902484736 6
0.89733833333981 12
0.9140718683562705 10
0.6160700286845627 3
0.9544537106815353 9
0.5317072331806809 3
0.8406739820427923 7
0.357761221576466 0
0.1328974587

...

0.29508650274723847 10
0.8187782776595897 4
0.17595454800545116 3
0.1899635698091297 11
0.6843723974731261 10
0.7492303538373051 11
0.3487074874798231 3
0.23465112721896858 8
0.47472894845886243 0
0.2771970092705667 7
0.6563838752355923 5
0.1411881727664064 3
0.2972125133557841 7
0.8377046766859569 0
0.3695359302653246 1
0.5593066408337346 7
0.9550082493561747 10
0.3286808517169935 12
0.795709381404585 2
0.9999257875924115 6
0.9261720798960676 3
0.38543717199056077 4
0.9708646282448901 9
0.6815770907505961 7
0.9017107689905544 4
0.9838805197762308 6
0.9808334674423889 12
0.9536982627571847 7
0.7003344320204183 2
0.7797217276294165 0
0.173233220190517 12
0.32848745634246485 2
0.9746762667857556 7
0.2797255385574454 1
0.7910606502635756 2
0.9719078791047006 12
0.808701307170583 0
0.525219690271872 7
0.8385556387069107 0
0.8809640694481203 3
0.6141208016260199 3
0.1747288951174401 1
0.9663301126557095 10
0.13845853078275944 2
0.9463520152384169 7
0.7184253610516361 6
0.20638499460368204 1

0.2312894202746384 12
0.999890202466237 6
0.6602968214801135 0
0.23241462845320396 11
0.2412301722394492 1
0.1884271957157294 4
0.5900443007186451 9
0.2595969826930194 2
0.4254443894517588 0
0.26807241702930124 1
0.6274282801770124 7
0.2655098670333153 6
0.2356208217878214 5
0.9536762834750808 9
0.8401615040540764 0
0.1759749517243195 4
0.7855796737230983 4
0.7080136797860237 4
0.19935756975819852 0
0.19820771269793155 0
0.8294296354740266 6
0.20034808532814954 1
0.21415488483722836 2
0.9200423571891163 5
0.13456606588709927 10
0.8966399559211266 9
0.5222831377081832 9
0.3064771526094283 4
0.20242918139371802 1
0.10582912385701629 5
0.9940767051281801 7
0.5813284758677842 7
0.24215515112854166 5
0.19508731489735245 8
0.17725129775103704 11
0.9085511421128383 9
0.24345796660029698 6
0.9843584930069399 0
0.44669744871100237 3
0.19683041156321432 7
0.5202372364087942 3
0.23303093832425578 1
0.649297658416359 8
0.7708587159259577 9
0.31589655735941463 4
0.3000473245324044 5
0.9593489396368

0.30557838364582485 8
0.4374815857844077 4
0.9555319419386635 7
0.5946788542563297 12
0.3593493146643572 5
0.6443645827393992 8
0.33561897987477335 2
0.4373415168976964 1
0.8697985629424849 12
0.3484559874113573 1
0.22460016994420548 5
0.5607327644402573 2
0.4961011965460678 12
0.9953807945321707 6
0.988234816978403 11
0.15515730911427117 8
0.27426828225006095 11
0.1751721146332605 0
0.5224570232729926 8
0.31779342133561583 3
0.810741193929714 12
0.5805161383827727 11
0.15310287347227056 0
0.1301725968353982 1
0.8188159058870592 7
0.10773118542844254 1
0.7372401241889611 10
0.1888410000956015 1
0.7661235329654461 11
0.5304973994736022 1
0.9991453369088982 6
0.19487614275269857 2
0.13906570806521715 7
0.14745510969257408 5
0.26738052472861634 5
0.1420548169547963 5
0.35275631860150153 3
0.4129689748284197 6
0.27087419891965436 1
0.6077168686585931 7
0.45326800204621503 3
0.8366416722444999 10
0.7082875450942534 6
0.7406037527185597 4
0.5353967535044044 7
0.1553762380331171 3
0.653497921

0.12671344257461273 11
0.36711623995911924 10
0.22097481560377508 12
0.8907636452310914 9
0.25063676805773794 4
0.9989698734069863 6
0.3516676759577182 0
0.20048728046305994 10
0.22821175744047512 0
0.4979806843385007 5
0.7044317768607607 3
0.21935096127734574 7
0.14760457751407927 5
0.4415135585728327 3
0.5984558576790394 8
0.8489182723866486 12
0.607747706478532 5
0.950255483418471 2
0.2634949024043911 0
0.6676337794260445 3
0.9814028030453152 3
0.7778988367286748 0
0.22939588056134358 7
0.8225411717243877 3
0.27065136929744565 1
0.1613994208784934 5
0.1925848911343528 11
0.4741682257460756 1
0.6376008089895643 4
0.33980342648355744 7
0.33112188485514443 11
0.3492993664194518 11
0.3030565412006565 5
0.1975723603288356 9
0.4049494067065802 0
0.5666223929719075 3
0.9835769108945459 7
0.3789879592619574 11
0.7528928461091341 5
0.8358878147501665 12
0.2611614840300642 8
0.7806293575656369 8
0.23808511927945195 4
0.8463894107263914 6
0.985208806376673 0
0.9861067952694752 0
0.704444123195

0.1901543615191731 4
0.35245851652559845 11
0.8551851088294894 1
0.836909479949157 6
0.942048910514444 7
0.1926377332231968 1
0.5179171529629669 6
0.8722871287040714 5
0.8128552303727341 6
0.5856721522339093 10
0.186370132619796 5
0.1434290254074053 0
0.9243493288897288 12
0.35075802595419087 3
0.19984767330469372 5
0.8744917970680172 9
0.35798383829730174 4
0.31613321954412277 6
0.16958923721962413 1
0.6294415825214261 0
0.31278241607708884 9
0.2355579572856376 0
0.9857067199958416 4
0.9095986572208035 4
0.23360056064567955 7
0.13288220972575437 2
0.11839888654382313 1
0.8366849664014113 4
0.5809928197437967 7
0.6840638206822192 12
0.5657162140467863 4
0.44948045820710536 10
0.4562166170434207 2
0.5037446654361097 2
0.6646213303752311 0
0.9688512846411087 11
0.14287893233548365 8
0.44155526295357894 10
0.6785471589642383 12
0.6111242182274172 3
0.9647014732473436 9
0.9934768574577422 6
0.38483761261056726 5
0.8048152226520481 0
0.789432743975223 10
0.18191026359235601 12
0.26856122690

In [112]:
real_test = pd.read_csv('data/test.csv')
real_test['category'] = results
real_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26260 entries, 0 to 26259
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   oid       26260 non-null  int64  
 1   text      26260 non-null  object 
 2   category  9818 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 615.6+ KB


In [113]:
real_test = real_test.dropna()
real_test['category'] = real_test['category'].astype(int)
real_test['category'] = real_test.apply(lambda x: list(category_index.keys())[list(category_index.values()).index(x['category'])], axis = 1)


answer = real_test.drop(['text'], axis=1)

In [114]:
answer = pd.DataFrame({'category': real_test['category'], 'oid':real_test['oid'] })
answer.to_csv('data/final_answer65_tf.csv', index=False)
real_test.to_csv('data/my_full_answer65_tf.csv', index=False)